# ML-5333 Project 2 - MLB Data Collection

In [2]:
import pandas as pd
import numpy as np
import baseball_scraper
import pybaseball
import statsapi
import mlbgame

In [ ]:
if __name__ == "__main__":

    test = statsapi.schedule(start_date="01/01/2019", end_date="12/31/2019", team=143)
    print(test.columns)

In [18]:
from pybaseball import schedule_and_record
data = schedule_and_record(2018, "NYY")